# Creating and Using a Custom Job with phospho Lab

In this notebook, we are going to:
- define our own job to be runned with phospho. In this example, it will be a regex text on the name of a city.
- add it to our phopho lab workload
- running this job on some messages

In [13]:
!pip install -q "phospho[lab]" 

## Creating our own REGEX job

In [11]:
from phospho import lab
from typing import List
import re

def my_custom_job(message: lab.Message, forbidden_words: List) -> lab.JobResult:
    """
    For each each message, me will check if the forbidden words are present in the message.
    The function will return a JobResult with a boolean value 
    (True if one of the words is present, False otherwise).
    """

    pattern = r'\b(' + '|'.join(re.escape(word) for word in forbidden_words) + r')\b'

    # Use re.search() to check if any of the words are in the text
    if re.search(pattern, message.content):
        result = True
    else:
        result = False
    
    return lab.JobResult(
        job_id="my_custom_job",
        result_type=lab.ResultType.bool,
        value=result,
    )

In [8]:
# Let's test the function
result = my_custom_job(lab.Message(content="I love my cat", sender="user"), ["cat", "dog"])
print(f"Result: {result.value} (expected: True)")

Result: True (expected: True)


## Adding it to our lab workload

In [10]:
# Create a workload in our lab
workload = lab.Workload()

# Add our job to the workload
workload.add_job(
    lab.Job(
        id="regex_check",
        job_function=my_custom_job, # We add our custom job function here
        config=lab.JobConfig(
            forbidden_words=["cat", "dog"]
        ),
    )
)

Now that everything is setup, we can run our job on messages and see if they match our regex.

In [12]:
await workload.async_run(
    messages=[
        # No forbiden word is present.
        lab.Message(
            id="message_1",
            content="I like elephants.",
        ),
        # One forbiden word is present.
        lab.Message(
            id="message_2",
            content="I love my cat.",
        ),
        # Both forbidden words present.
        lab.Message(
            id="message_3",
            content="I love my cat and my dog.",
        ),
    ]
)

# Let's see the results
for i in range(1, 4):
    print(
        f"In message {i}, a forbidden word was detected: {workload.results['message_'+str(i)]['regex_check'].value}"
    )

In message 1, a forbidden word was detected: False
In message 2, a forbidden word was detected: True
In message 3, a forbidden word was detected: True


## Running the job on some messages generated by OpenAI GPT-3.5 Turbo

In [16]:
!pip install -q python-dotenv openai

In [15]:
# Load and check env variables
from dotenv import load_dotenv

load_dotenv()

from phospho import config

assert (
    config.OPENAI_API_KEY is not None
), "You need to set the OPENAI_API_KEY environment variable"

In [24]:
# Generate a poem about animals, let's see if the forbidden words are present

from openai import OpenAI
client = OpenAI(api_key=config.OPENAI_API_KEY)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a brilliant poet."},
    {"role": "user", "content": "Write me a poem about animals."},
  ],
  temperature=0.9,
)

poem_1 = completion.choices[0].message.content

print("First poem:")
print(poem_1[:200] + "..." if len(poem_1) > 200 else poem_1)

# Let's do another one, increasing the probability of the forbidden words
# Actually have no idea if the forbidden words will be present
# You can re-run this cell to get different results
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a brilliant poet."},
    {"role": "user", "content": "Write me a poem about animals. Name some animals."},
  ],
  temperature=0.9,
)

poem_2 = completion.choices[0].message.content

print("")
print("Second poem:")
print(poem_2[:200] + "..." if len(poem_2) > 200 else poem_2)

# Let's do a final one, where the forbidden words should be present
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a brilliant poet."},
    {"role": "user", "content": "Write me a poem about cat and dogs."},
  ],
  temperature=0.9,
)

poem_3 = completion.choices[0].message.content

print("")
print("Third poem:")
print(poem_3[:200] + "..." if len(poem_3) > 200 else poem_3)


First poem:
In the heart of the forest deep and wild,
Where creatures of all kinds roam free,
There lies a beauty pure and undefiled,
In the animals that dwell with glee.

The graceful deer with eyes so kind,
Mov...

Second poem:
In a world of wonder, where nature reigns,
Among the creatures that roam the plains,
The mighty lion with golden mane,
Stalks his prey, a master of the game.

The graceful dolphin in ocean's deep,
Dan...

Third poem:
In a world where cats and dogs meet,
A dance of harmony, so sweet.
With whiskers twitching and tails wagging,
Their friendship everlasting, never sagging.

The cat, so sly and sleek and wise,
Inquisit...


In [25]:
await workload.async_run(
    messages=[
        # Low probability of forbidden words.
        lab.Message(
            id="poem_1",
            content=poem_1,
        ),
        # High probability of forbidden words.
        lab.Message(
            id="poem_2",
            content=poem_2,
        ),
        # Forbidden words should be present.
        lab.Message(
            id="poem_3",
            content=poem_3,
        ),
    ]
)

# Let's see the results
for i in range(1, 4):
    print(
        f"In message {i}, a forbidden word was detected: {workload.results['poem_'+str(i)]['regex_check'].value}"
    )

In message 1, a forbidden word was detected: False
In message 2, a forbidden word was detected: False
In message 3, a forbidden word was detected: True


## Going further

Try implementing more complex jobs, such as a job that uses a machine learning model to classify messages, or a job that uses a NLP model to extract entities from messages.

And why note opening a pull request to add your job to the phospho lab workload? We would love to see what you come up with!

You want to have such analysis on your own LLM app, in real time? Check out the cloud hosted version of phospho, available on [phospho.ai](https://phospho.ai)
